<a href="https://colab.research.google.com/github/ShreshthSaxena/Text-scraping-and-mining-Wikipedia/blob/master/Text_scraping_and_mining_(Wikipedia)-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Imports

In [1]:
!pip install wikipedia
import wikipedia

  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd

## Getting the list of top 100 countries by GDP

In [0]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)')
b = BeautifulSoup(html.text, 'lxml')
links = []

#print(b.prettify())

In [0]:
My_table = b.find('table',{'class':'wikitable sortable'})
#My_table

In [5]:
links = My_table.findAll('a')
links = links[2:111]
links[:5]

[<a href="/wiki/United_States" title="United States">United States</a>,
 <a href="/wiki/European_Union" title="European Union">European Union</a>,
 <a href="#cite_note-EU_note-25">[n 1]</a>,
 <a href="/wiki/China" title="China">China</a>,
 <a href="#cite_note-China-THM-26">[n 2]</a>]

In [6]:
Countries = []
for link in links:
  Countries.append(link.get('title'))
  
Countries = list(filter(None, Countries)) 
Countries[:10]
# df = pd.DataFrame(Countries, columns=['Countries'])
# df

['United States',
 'European Union',
 'China',
 'Japan',
 'Germany',
 'United Kingdom',
 'France',
 'India',
 'Italy',
 'Brazil']

**creating links to wiki page of each country**

In [8]:
page_links = ['https://en.wikipedia.org/wiki/Economy_of_' + i.replace(" ",'_') for i in Countries]
page_links[69] = 'https://en.wikipedia.org/wiki/Economy_of_the_Dominican_Republic'
page_links[:10]

['https://en.wikipedia.org/wiki/Economy_of_United_States',
 'https://en.wikipedia.org/wiki/Economy_of_European_Union',
 'https://en.wikipedia.org/wiki/Economy_of_China',
 'https://en.wikipedia.org/wiki/Economy_of_Japan',
 'https://en.wikipedia.org/wiki/Economy_of_Germany',
 'https://en.wikipedia.org/wiki/Economy_of_United_Kingdom',
 'https://en.wikipedia.org/wiki/Economy_of_France',
 'https://en.wikipedia.org/wiki/Economy_of_India',
 'https://en.wikipedia.org/wiki/Economy_of_Italy',
 'https://en.wikipedia.org/wiki/Economy_of_Brazil']

In [0]:
# html = requests.get('https://en.wikipedia.org/wiki/Economy_of_India')
# soup = BeautifulSoup(html.text, 'lxml')
# #text = soup.get_text()
# print(soup.title)

# print(wikipedia.page('Economy of Italy').content)
# #print(te[:te.index("== See also")])

## Getting Words from wiki links

In [0]:
keywords = ['tariffs',
            'GDP',
            'unemployment',
            'industry',
            'agriculture',
            'net_worth',
            'insurance',
            'health_care',
            'trade',
            'venture_capital']

In [0]:
content = []
for i in range(105):
    print('Collecting words from '+ page_links[i])
    #html = requests.get(link)
    #b = BeautifulSoup(html.text, 'lxml')
    text = wikipedia.page('Economy of '+ Countries[i]).content
    if "== See also" in text:
      text = text[:text.index("== See also")]
    else:
      text = text[:text.index("== External links")]
    content.append(text)
    
len(content)


105

## Pre processing words

In [0]:
#Alternative tokenization
# import re
# words = re.split(r'\W+', content[0])

# words

In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import string

nltk.download('stopwords')
nltk.download('punkt')
st = LancasterStemmer()

def tokenize(text):
  tokens = word_tokenize(text)
  tokens = [word.lower() for word in tokens]

  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]

  words = [word for word in stripped if word.isalpha()]
  
  words = [st.stem(word) for word in words]

  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  words = list(set(words))
  return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Calculating similarities

Similarities are stored in the multi index matrix -> a 

Decrease threshold to increase the nuumber of related keywords

In [0]:
print(keywords)

['tariffs', 'GDP', 'unemployment', 'industry', 'agriculture', 'net worth', 'insurance', 'health care', 'trade', 'venture capital']


In [0]:
sim = pd.DataFrame(columns = ['Given Keyword','Related Keyword', 'Closeness'])
sim

,Given Keyword,Related Keyword,Closeness


In [0]:
res = "".join(content)
len(res)

3268569

In [0]:
keywords[5].replace('_',' ')

'net worth'

In [0]:
from nltk.corpus import wordnet
nltk.download('wordnet')
s = 0

for word1 in keywords:
    wordFromList1 = wordnet.synsets(word1)
    words = tokenize(res)
    for word2 in words:
      wordFromList2 = wordnet.synsets(word2)
      if wordFromList1 and wordFromList2:
        s = wordFromList1[0].wup_similarity(wordFromList2[0])
        if s and s > 0.7:                                                                             ####CHANGE THRESHOLD FOR INCREASING?DECREASING NUMBER OF RELATED KEYWORDS
          print(word1, word2, s)
          sim = sim.append({'Given Keyword':word1, 'Related Keyword':word2, 'Closeness':s}, ignore_index=True)
              
              
# for word in content[0].split():
#     print(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
tariffs levy 0.8235294117647058
tariffs tariff 1.0
tariffs surtax 0.8
tariffs tax 0.8888888888888888
tariffs vat 0.8571428571428571
GDP gdps 1.0
GDP merit 0.7692307692307693
GDP gnp 0.8571428571428571
GDP gdp 1.0
unemployment halt 0.7272727272727273
unemployment alert 0.7272727272727273
unemployment hum 0.7272727272727273
unemployment comfort 0.7272727272727273
unemployment summit 0.7272727272727273
unemployment discord 0.7272727272727273
unemployment thing 0.7272727272727273
unemployment mood 0.7272727272727273
unemployment employ 0.8
unemployment freedom 0.8
unemployment flaw 0.7272727272727273
unemployment strife 0.7272727272727273
unemployment ready 0.7272727272727273
unemployment malady 0.7272727272727273
unemployment defect 0.7272727272727273
unemployment dark 0.7272727272727273
unemployment wild 0.8
unemployment affect 0.7272727272727273
industry firms 0.75
industry firm 0.

In [0]:
sim = sim[sim.Closeness != 1]
a=sim.set_index(['Given Keyword','Related Keyword'])

In [0]:
sim

,Given Keyword,Related Keyword,Closeness
0,tariffs,levy,0.823529
2,tariffs,surtax,0.800000
3,tariffs,tax,0.888889
4,tariffs,vat,0.857143
6,GDP,merit,0.769231
7,GDP,gnp,0.857143
9,unemployment,halt,0.727273
10,unemployment,alert,0.727273
11,unemployment,hum,0.727273
12,unemployment,comfort,0.727273


Similarities to the keyword **'industry'**

In [0]:
a.loc['industry']

,similarity
word,
firms,0.750000
firm,0.750000
corporations,0.705882
collective,0.800000
parties,0.714286
association,0.714286
organization,0.769231
unions,0.714286
business,0.800000
